In [ ]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob, os
from scipy.signal import savgol_filter
import numpy as np
import seaborn as sn
%load_ext tensorboard
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import load_model
import tf2onnx
import onnx
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import dump, load
from keras.models import Sequential
from keras.layers import Input, LSTM,Dense, GaussianNoise, BatchNormalization, Activation, PReLU, Dropout
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
from keras.optimizers import nadam_v2, rmsprop_v2, adamax_v2
from keras.activations import silu,swish, linear, tanh,leaky_relu, sigmoid, relu,elu, selu, gelu, linear
from keras.regularizers import l1, l2, L1L2
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def special_preprocess(data: np.ndarray):
    delta_t_spindel_bett= data[:,1] - data[:,0]
    delta_t_spindel_bett= delta_t_spindel_bett.reshape((-1,1))
    result= np.concatenate(( data[:,2:5], delta_t_spindel_bett), axis=1)
    return result

def rounding(signals: np.ndarray):
    return np.round(signals,3)


In [ ]:
### set parameters


main_path= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/feature_generation/'
generated_feature='t_motor'
source_dir= main_path+ generated_feature+ '/problem_with_t_motor/'
target_dir=source_dir+'/corrected/'
main_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\t_motor\\assets\\'
model_dir= main_dir + 'model\\'
preprocessor_dir= main_dir + 'preprocessor\\'
model_name= 'model.h5'
scaler= StandardScaler(with_mean= True, with_std= True)
minmax= MinMaxScaler(feature_range=(-1,1))
pca= PCA(n_components=1)
##Data Parameters
selected_Columns= ['t_spindel','drhz']#,'M8','M121','M127','M7','t_kuehlung']#['t_motor','DRZ']
window=15
shift= 1
proposed_pipline= Pipeline(steps=[('stdscaler', scaler), ('pca', pca)])# Pipeline(steps=[('stdscaler', scaler),('special', FunctionTransformer(rounding))])# 
preprocessor_name= "preprocessor.p"

In [ ]:
def generateDataSource(signal_input= None, input_columns: list = [], output_length: int=1, signal_output= None, window= 1 , shift= 1):
    Signal_Length = signal_input.shape[0]
    num_samples= int((Signal_Length - window +1 ) / shift)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y= np.zeros(shape=(num_samples, output_length, 1))
    for i in range (num_samples):
        x[i]= signal_input[i * shift : i * shift + window]
        y[i]= signal_output[i * shift + window - 1 : i * shift + window - 1+ output_length]
    return x, y

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')

In [ ]:
columns= ['t_bett', 't_spindel',
       't_welle', 't_schlitten', 'drhz', 'z_schlitten_mk', 'z_schlitten_ok',
       'z_hals_mk', 'z_hals_ok', 'z_spindel_mk', 'z_spindel_ok',
       't_motor', 'pred_t_motor']

In [ ]:
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
#model= load_model(str(model_dir), compile= False)
model_dir='G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\t_motor\\assets\\model\\model.h5'
model= load_model(model_dir, compile= False)
print('Model is loaded successfully ')
os.chdir(source_dir)
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df= pd.read_csv(file)
    signals= df
    signals= signals[selected_Columns].to_numpy()
    signals= pipeline.transform(signals)
    output= df['t_motor']
    partitions, target= generateDataSource(signal_input= signals, input_columns= selected_Columns, output_length= 1,signal_output= output, window= window, shift= shift)
    print('input: ', partitions.shape, ' output:', target.shape)
    pred= model.predict(partitions)
    print (df.columns)
    df= df[window-1:]
    print (df.shape)
    print (pred.shape)
    df['t_motor']= pred
    df.to_csv(path_or_buf=str(target_dir+'corrected_'+file))
    #df2= pd.DataFrame({'targets': np.reshape(target,-1), 'pred': np.reshape(pred,-1)})
    fig= ex.line(df,y=['t_motor'])
    #fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
    fig.show()

In [ ]:
### Calculate predictions

In [ ]:
### visualize Results